In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm, t
from statsmodels.tsa.arima.model import ARIMA

## Problem 1

In [2]:
np.random.seed(42)
sigma = 1
P_t_minus_1 = 100
n = 1000

r_t_samples = np.random.normal(0, sigma, n)

classical_returns = P_t_minus_1 + r_t_samples
arithmetic_returns = P_t_minus_1 * (1 + r_t_samples)
log_returns = P_t_minus_1 * np.exp(r_t_samples)

classical_mean = np.mean(classical_returns)
classical_std = np.std(classical_returns)
arithmetic_mean = np.mean(arithmetic_returns)
arithmetic_std = np.std(arithmetic_returns)
log_mean = np.mean(log_returns)
log_std = np.std(log_returns)

results = {
    "Classical Returns": (classical_mean, classical_std),
    "Arithmetic Returns": (arithmetic_mean, arithmetic_std),
    "Log Returns": (log_mean, log_std)
}
results

{'Classical Returns': (100.01933205582233, 0.9787262077473542),
 'Arithmetic Returns': (101.93320558223256, 97.87262077473542),
 'Log Returns': (168.23341551794823, 244.4927936930635)}

## Problem 2

In [3]:
prices = pd.read_csv("DailyPrices.csv")

In [4]:
def return_calculate(prices, method="DISCRETE", date_column="Date"):
    if date_column not in prices.columns:
        raise ValueError(f"dateColumn: {date_column} not in DataFrame: {prices.columns}")

    vars = [col for col in prices.columns if col != date_column]
    n_vars = len(vars)
    p = prices[vars].values
    n, m = p.shape

    p2 = np.zeros((n-1, m))
    for i in range(n-1):
        for j in range(m):
            p2[i, j] = p[i+1, j] / p[i, j]

    if method.upper() == "DISCRETE":
        p2 -= 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p2)
    else:
        raise ValueError(f"method: {method} must be in ('LOG','DISCRETE')")

    out = pd.DataFrame(p2, columns=vars)
    out.insert(0, date_column, prices[date_column].values[1:])

    return out

In [5]:
arithmetic_returns = return_calculate(prices, "DISCRETE", "Date")
arithmetic_returns = arithmetic_returns.iloc[:-1]
arithmetic_returns

,Date,SPY,AAPL,MSFT,AMZN,NVDA,GOOGL,TSLA,GOOG,BRK-B,...,CI,ETN,SLB,PGR,SCHW,LRCX,ZTS,C,BSX,AMT
0,2022-09-02,-0.010544,-0.013611,-0.016667,-0.002425,-0.020808,-0.017223,-0.025076,-0.016915,-0.016854,...,-0.001180,-0.010593,0.033107,-0.010428,-0.019242,-0.004236,-0.015244,0.001846,-0.012198,-0.026355
1,2022-09-06,-0.003773,-0.008215,-0.010974,-0.010980,-0.013336,-0.009643,0.015581,-0.011042,-0.003890,...,-0.004641,0.008449,-0.014118,0.000572,0.001848,-0.008019,-0.000892,-0.012695,-0.002717,0.013275
2,2022-09-07,0.017965,0.009254,0.019111,0.026723,0.018795,0.024717,0.033817,0.027912,0.016089,...,0.016652,0.020295,-0.008030,0.038537,0.018731,0.012279,0.022698,0.008503,0.026994,0.020930
3,2022-09-08,0.006536,-0.009618,0.001666,0.002626,0.020126,-0.009776,0.019598,-0.009595,0.008184,...,0.002448,0.013945,0.029951,0.015880,0.019083,0.016574,-0.011908,0.026116,0.029901,0.008362
4,2022-09-09,0.015535,0.018840,0.022977,0.026575,0.028377,0.020945,0.036023,0.021568,0.008576,...,0.007327,0.017244,0.038774,-0.004179,0.018863,0.026460,0.036721,0.015431,0.005385,-0.000306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,2023-09-15,-0.012048,-0.004154,-0.025037,-0.029920,-0.036879,-0.005069,-0.005977,-0.004964,-0.004438,...,-0.000318,-0.020302,-0.016558,-0.005947,-0.025770,-0.050759,-0.013626,-0.009968,-0.000378,-0.005191
260,2023-09-18,0.000586,0.016913,-0.003513,-0.002920,0.001503,0.005895,-0.033201,0.004772,0.006986,...,0.007485,0.006938,0.010399,0.013118,-0.006183,0.020125,-0.003329,-0.001639,0.001890,-0.003386
261,2023-09-19,-0.002074,0.006181,-0.001246,-0.016788,-0.010144,-0.001230,0.004599,-0.000936,0.000135,...,-0.002453,-0.013644,-0.012743,0.013589,-0.002247,-0.016519,0.012970,0.000938,0.000566,-0.012087
262,2023-09-20,-0.009193,-0.019992,-0.023977,-0.017002,-0.029435,-0.031150,-0.014672,-0.030541,-0.009879,...,0.009450,-0.006986,-0.010591,0.001544,-0.018361,-0.010062,-0.002748,-0.008903,0.020177,0.000282


In [6]:
mean_meta = arithmetic_returns['META'].mean()
arithmetic_returns['META'] = arithmetic_returns['META'] - mean_meta
arithmetic_returns['META']

0     -0.033234
1     -0.013858
2      0.008914
3      0.007657
4      0.040994
         ...   
259   -0.039358
260    0.004704
261    0.005574
262   -0.020456
263   -0.015903
Name: META, Length: 264, dtype: float64

In [7]:
# Using a normal distribution
# Assuming the confidence level is 0.95
var_normal = 1.65 * arithmetic_returns['META'].std()
print(var_normal)

0.05455326925723376


In [8]:
# Using a normal distribution with an Exponentially Weighted variance
def calculate_ewma_variance(data, lambda_EW):
    ewma_variance = np.zeros(len(data))
    ewma_variance[0] = data.var()  # Initial variance is the sample variance
    for t in range(1, len(data)):
        ewma_variance[t] = lambda_EW * ewma_variance[t-1] + (1 - lambda_EW) * (data.iloc[t-1] - data.iloc[:t].mean())**2
    return ewma_variance

lambda_EW = 0.94
variance_EW_custom = calculate_ewma_variance(arithmetic_returns['META'], lambda_EW)
std_dev_EW_custom = np.sqrt(variance_EW_custom)
var_normal_EW_custom = 1.65 * std_dev_EW_custom[-1]
print(var_normal_EW_custom)

0.03106754744474217


In [9]:
# Using a MLE fitted T distribution
params = t.fit(arithmetic_returns['META'])
var_mle_t = t.ppf(0.95, *params)
print(var_mle_t)

0.04112246904952898


In [10]:
# Using a fitted AR(1) model
model = ARIMA(arithmetic_returns['META'], order=(1, 0, 0))
results = model.fit()
n_simulations = 1000
forecast_horizon = 1
simulated_values = []
for _ in range(n_simulations):
    simulated_series = results.simulate(nsimulations=forecast_horizon)
    simulated_values.append(simulated_series.iloc[-1])

var_ar1 = -np.quantile(simulated_values, 0.05)

print(var_ar1)

0.0520304404475868


In [11]:
# Using a Historic Simulation
sorted_returns = arithmetic_returns['META'].sort_values()
var_historic = sorted_returns.quantile(0.95)
print(var_historic)

0.04121586193570658


## Problem 3

In [12]:
portfolio = pd.read_csv('Portfolio.csv')
lambda_EW = 0.94

portfolio_A = portfolio[portfolio['Portfolio'] == 'A']['Stock'].tolist()
portfolio_B = portfolio[portfolio['Portfolio'] == 'B']['Stock'].tolist()
portfolio_C = portfolio[portfolio['Portfolio'] == 'C']['Stock'].tolist()

holdings_A = portfolio[portfolio['Portfolio'] == 'A']['Holding'].tolist()
holdings_B = portfolio[portfolio['Portfolio'] == 'B']['Holding'].tolist()
holdings_C = portfolio[portfolio['Portfolio'] == 'C']['Holding'].tolist()

portfolio_A_prices = prices[portfolio_A].values
portfolio_B_prices = prices[portfolio_B].values
portfolio_C_prices = prices[portfolio_C].values

def calculate_portfolio_returns(portfolio_stocks, portfolio_holdings):
    portfolio_prices = prices[portfolio_stocks].values
    portfolio_returns = (portfolio_prices[1:] - portfolio_prices[:-1]) / portfolio_prices[:-1]
    return portfolio_returns

portfolio_A_returns = calculate_portfolio_returns(portfolio_A, holdings_A)
portfolio_B_returns = calculate_portfolio_returns(portfolio_B, holdings_B)
portfolio_C_returns = calculate_portfolio_returns(portfolio_C, holdings_C)

In [13]:
def calculate_ewma_covariance(returns, lambda_EW):
    n_obs, n_stocks = returns.shape
    ewma_covariance = np.zeros((n_stocks, n_stocks))
    ewma_covariance = np.cov(returns.T, aweights=np.array([(lambda_EW)**(n_obs-1-i) for i in range(n_obs)]))

    return ewma_covariance

cov_A = calculate_ewma_covariance(portfolio_A_returns, lambda_EW)
cov_B = calculate_ewma_covariance(portfolio_B_returns, lambda_EW)
cov_C = calculate_ewma_covariance(portfolio_C_returns, lambda_EW)

total_returns = np.concatenate((portfolio_A_returns, portfolio_B_returns, portfolio_C_returns), axis=1)

def calculate_total_ewma_covariance(returns, lambda_EW):
    n_obs, n_stocks = returns.shape
    ewma_covariances = np.cov(returns.T, aweights=np.array([(lambda_EW)**(n_obs-1-i) for i in range(n_obs)]))
    return ewma_covariances

total_cov = calculate_total_ewma_covariance(total_returns, lambda_EW)

portfolio_A_values = portfolio_A_prices[-1,:] * holdings_A
portfolio_B_values = portfolio_B_prices[-1,:] * holdings_B
portfolio_C_values = portfolio_C_prices[-1,:] * holdings_C

total_portfolio_value = np.sum(portfolio_A_values) + np.sum(portfolio_B_values) + np.sum(portfolio_C_values)

portfolio_A_var = 1.65 * np.sqrt(np.dot(portfolio_A_values, np.dot(cov_A, portfolio_A_values)))
portfolio_B_var = 1.65 * np.sqrt(np.dot(portfolio_B_values, np.dot(cov_B, portfolio_B_values)))
portfolio_C_var = 1.65 * np.sqrt(np.dot(portfolio_C_values, np.dot(cov_C, portfolio_C_values)))
total_var = 1.65 * np.sqrt(np.dot(np.concatenate((portfolio_A_values, portfolio_B_values, portfolio_C_values)), np.dot(total_cov, np.concatenate((portfolio_A_values, portfolio_B_values, portfolio_C_values)))))

print(f"Portfolio A VaR: ${portfolio_A_var:.2f}")
print(f"Portfolio B VaR: ${portfolio_B_var:.2f}")
print(f"Portfolio C VaR: ${portfolio_C_var:.2f}")
print(f"Total VaR: ${total_var:.2f}")

Portfolio A VaR: $15253.97
Portfolio B VaR: $7765.47
Portfolio C VaR: $17933.67
Total VaR: $38091.10


In [14]:
def calculate_portfolio_returns_log(portfolio_stocks, portfolio_holdings):
    portfolio_prices = prices[portfolio_stocks].values
    portfolio_returns = np.log(portfolio_prices[1:] / portfolio_prices[:-1])
    return portfolio_returns

In [15]:
portfolio_A_returns_log = calculate_portfolio_returns_log(portfolio_A, holdings_A)
portfolio_B_returns_log = calculate_portfolio_returns_log(portfolio_B, holdings_B)
portfolio_C_returns_log = calculate_portfolio_returns_log(portfolio_C, holdings_C)

cov_A_log = calculate_ewma_covariance(portfolio_A_returns_log, lambda_EW)
cov_B_log = calculate_ewma_covariance(portfolio_B_returns_log, lambda_EW)
cov_C_log = calculate_ewma_covariance(portfolio_C_returns_log, lambda_EW)

total_returns_log = np.concatenate((portfolio_A_returns_log, portfolio_B_returns_log, portfolio_C_returns_log), axis=1)
total_cov_log = calculate_total_ewma_covariance(total_returns_log, lambda_EW)

portfolio_A_var_log = 1.65 * np.sqrt(np.dot(portfolio_A_values, np.dot(cov_A_log, portfolio_A_values)))
portfolio_B_var_log = 1.65 * np.sqrt(np.dot(portfolio_B_values, np.dot(cov_B_log, portfolio_B_values)))
portfolio_C_var_log = 1.65 * np.sqrt(np.dot(portfolio_C_values, np.dot(cov_C_log, portfolio_C_values)))
total_var_log = 1.65 * np.sqrt(np.dot(np.concatenate((portfolio_A_values, portfolio_B_values, portfolio_C_values)), np.dot(total_cov_log, np.concatenate((portfolio_A_values, portfolio_B_values, portfolio_C_values)))))

print(f"Portfolio A VaR: ${portfolio_A_var_log:.2f}")
print(f"Portfolio B VaR: ${portfolio_B_var_log:.2f}")
print(f"Portfolio C VaR: ${portfolio_C_var_log:.2f}")
print(f"Total VaR: ${total_var_log:.2f}")

Portfolio A VaR: $15289.78
Portfolio B VaR: $7799.41
Portfolio C VaR: $17891.81
Total VaR: $38158.29
